In [1]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter
from scipy.io import wavfile
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from skimage.transform import resize
from torchvision import transforms
import torchaudio.transforms as ta_transforms
import math
import torchaudio
import time
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
PATH=os.getenv("HOME")+"/aiffel/ECG_data/physionet.org/files/circor-heart-sound/1.0.3/training_data"
SAMPLE_RATE = 4000
HOP_LENGTH = 40

In [ ]:
# txt파일 불러오기
file_list = os.listdir(PATH)
txt_list = [os.path.join(PATH, file) for file in file_list if file.endswith(".txt")]

# 환자 아이디를 훈련 데이터셋과 테스트 데이터셋으로 나눔
train_patient_txt, test_patient_txt = train_test_split(txt_list, test_size=0.9, random_state=42)

# 결과 출력
print("Train Patient IDs:", train_patient_txt[:3])
print("Test Patient IDs:", test_patient_txt[:3])

In [5]:
class Biquad:

  # pretend enumeration
    LOWPASS, HIGHPASS, BANDPASS, NOTCH, PEAK, LOWSHELF, HIGHSHELF = range(7)

    def __init__(self,typ, freq, srate, Q, dbGain = 0):
        types = {
            Biquad.LOWPASS : self.lowpass,
            Biquad.HIGHPASS : self.highpass,
            Biquad.BANDPASS : self.bandpass,
            Biquad.NOTCH : self.notch,
            Biquad.PEAK : self.peak,
            Biquad.LOWSHELF : self.lowshelf,
            Biquad.HIGHSHELF : self.highshelf
          }
        assert typ in types
        freq = float(freq)
        self.srate = float(srate)
        Q = float(Q)
        dbGain = float(dbGain)
        self.a0 = self.a1 = self.a2 = 0
        self.b0 = self.b1 = self.b2 = 0
        self.x1 = self.x2 = 0
        self.y1 = self.y2 = 0
        # only used for peaking and shelving filter types
        A = math.pow(10, dbGain / 40)
        omega = 2 * math.pi * freq / self.srate
        sn = math.sin(omega)
        cs = math.cos(omega)
        alpha = sn / (2*Q)
        beta = math.sqrt(A + A)
        types[typ](A,omega,sn,cs,alpha,beta)
        # prescale constants
        self.b0 /= self.a0
        self.b1 /= self.a0
        self.b2 /= self.a0
        self.a1 /= self.a0
        self.a2 /= self.a0

    def lowpass(self,A,omega,sn,cs,alpha,beta):
        self.b0 = (1 - cs) /2
        self.b1 = 1 - cs
        self.b2 = (1 - cs) /2
        self.a0 = 1 + alpha
        self.a1 = -2 * cs
        self.a2 = 1 - alpha

    def highpass(self,A,omega,sn,cs,alpha,beta):
        self.b0 = (1 + cs) /2
        self.b1 = -(1 + cs)
        self.b2 = (1 + cs) /2
        self.a0 = 1 + alpha
        self.a1 = -2 * cs
        self.a2 = 1 - alpha

    def bandpass(self,A,omega,sn,cs,alpha,beta):
        self.b0 = alpha
        self.b1 = 0
        self.b2 = -alpha
        self.a0 = 1 + alpha
        self.a1 = -2 * cs
        self.a2 = 1 - alpha

    def notch(self,A,omega,sn,cs,alpha,beta):
        self.b0 = 1
        self.b1 = -2 * cs
        self.b2 = 1
        self.a0 = 1 + alpha
        self.a1 = -2 * cs
        self.a2 = 1 - alpha

    def peak(self,A,omega,sn,cs,alpha,beta):
        self.b0 = 1 + (alpha * A)
        self.b1 = -2 * cs
        self.b2 = 1 - (alpha * A)
        self.a0 = 1 + (alpha /A)
        self.a1 = -2 * cs
        self.a2 = 1 - (alpha /A)

    def lowshelf(self,A,omega,sn,cs,alpha,beta):
        self.b0 = A * ((A + 1) - (A - 1) * cs + beta * sn)
        self.b1 = 2 * A * ((A - 1) - (A + 1) * cs)
        self.b2 = A * ((A + 1) - (A - 1) * cs - beta * sn)
        self.a0 = (A + 1) + (A - 1) * cs + beta * sn
        self.a1 = -2 * ((A - 1) + (A + 1) * cs)
        self.a2 = (A + 1) + (A - 1) * cs - beta * sn

    def highshelf(self,A,omega,sn,cs,alpha,beta):
        self.b0 = A * ((A + 1) + (A - 1) * cs + beta * sn)
        self.b1 = -2 * A * ((A - 1) + (A + 1) * cs)
        self.b2 = A * ((A + 1) + (A - 1) * cs - beta * sn)
        self.a0 = (A + 1) - (A - 1) * cs + beta * sn
        self.a1 = 2 * ((A - 1) - (A + 1) * cs)
        self.a2 = (A + 1) - (A - 1) * cs - beta * sn

  # perform filtering function
    def __call__(self,x):
        y = self.b0 * x + self.b1 * self.x1 + self.b2 * self.x2 - self.a1 * self.y1 - self.a2 * self.y2
        self.x2, self.x1 = self.x1, x
        self.y2, self.y1 = self.y1, y

        return y

  # provide a static result for a given frequency f
    def result(self,f):
        phi = (math.sin(math.pi * f * 2/(2.0 * self.srate)))**2
        return ((self.b0+self.b1+self.b2)**2 - \
        4*(self.b0*self.b1 + 4*self.b0*self.b2 + \
        self.b1*self.b2)*phi + 16*self.b0*self.b2*phi*phi) / \
        ((1+self.a1+self.a2)**2 - 4*(self.a1 + \
        4*self.a2 + self.a1*self.a2)*phi + 16*self.a2*phi*phi)

    def log_result(self,f):
        try:
            r = 10 * math.log10(self.result(f))
        except:
            r = -200
        return r

  # return computed constants
    def constants(self):
        return self.b0,self.b1,self.b2,self.a1,self.a2

In [6]:
class CustomDataset(Dataset):
    def __init__(self, path, txt_list,
                 target_size=(40, 2500),
                 th=25,
                 resizing=False,
                 filtering=False,
                 filter_hz=500):
        self.path = path
        self.txt_list = txt_list
        self.target_size = target_size
        self.th = int(th * SAMPLE_RATE / HOP_LENGTH)
        self.resizing = resizing
        self.filtering = filtering
        self.filter_hz = filter_hz

        self.get_file_list()

        self.delete_list=[]
        self.x = self.get_mel_spectrogram()
        self.y = self.get_label()
        self.delete_data()

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def get_file_list(self):
        self.heas = []
        self.wavs = []
        self.tsvs = []

        for path_txt in self.txt_list:
            with open(path_txt, "r") as f:
                P_id, n, sr = f.readline().split()
                for _ in range(int(n)):
                    _, hea, wav, tsv = f.readline().split()
                    self.heas.append(hea)
                    self.wavs.append(wav)
                    self.tsvs.append(tsv)
        self.heas.sort()
        self.wavs.sort()
        self.tsvs.sort()

    # torchaudio로 필터링 적용
    def apply_filter_torchaudio(self, audio, cutoff_hz):
        biquad = Biquad(typ=Biquad.LOWPASS, freq=cutoff_hz, srate=SAMPLE_RATE, Q=0.707)
        b0, b1, b2, a1, a2 = biquad.constants()
        b0 = torch.tensor(b0)
        b1 = torch.tensor(b1)
        b2 = torch.tensor(b2)
        a0 = 1
        a1 = torch.tensor(a1)
        a2 = torch.tensor(a2)
        filtered_audio = torchaudio.functional.biquad(audio, b0, b1, b2, a0, a1, a2)
        return filtered_audio

    def padding(self, spec, target_length, padding_value=0):
        pad_width = target_length - spec.shape[-1]
        padded_spec = torch.nn.functional.pad(spec, (0, pad_width, 0, 0), "constant", 0)
        return padded_spec

    def resize_spectrogram(self, spec, new_shape):
        resized_spec = transforms.functional.resize(img=spec, size=new_shape, antialias=None)
        return resized_spec

    # # 나누기 전 이미지에 대한 정규화를 위해 사용
    # def normalize_spectrogram(self, wav, spec):
    #     wav_max = np.max(np.array(wav))
    #     spec_max = np.max(np.array(spec))
    #     norm_max = spec_max / wav_max
    #     print(norm_max)
    #     normalized = spec / norm_max
    #     return normalized

    # 나누어진 개별 이미지의 최대값을 이용한 정규화
    def normalize_spectrogram2(self, spec):
        spec_max = np.max(np.array(spec))
        spec_min = np.min(np.array(spec))
        normalized = (spec-spec_min) / (spec_max-spec_min)
        return normalized

    # # torchvision을 사용한 정규화
    # def normalize_spectrogram3(self, spec):
    #     normalized = transforms.Normalize((0), (0.5))(spec)
    #     return normalized

    def get_mel_spectrogram(self):
        audio_list = []
        self.scale_list = []
        self.iter_list = []

        for path_wav in self.wavs:
            path = os.path.join(PATH, path_wav)

            # Torchaudio 이용하여 파일 로드
            x = torchaudio.load(path)[0]
            # Filtering
            if self.filtering is True:
                cutoff_frequency = self.filter_hz
                x = self.apply_filter_torchaudio(x, cutoff_frequency)
            ms = ta_transforms.MelSpectrogram(sample_rate=SAMPLE_RATE,
                                        n_fft=128,
                                        win_length=100,
                                        n_mels=40,
                                        hop_length=HOP_LENGTH)(x)

            # 스펙트로그램의 형태를 유지하면서 torchaudio로 불러왔을 때와 같이 0~1로 정규화
            # ms = self.normalize_spectrogram(x, ms)

            # 원본 wav의 길이가 th보다 길다면 Slicing
            if ms.shape[-1] > self.th:
                scale = 1
                num_splits = ms.shape[-1] // self.th    # wav길이 == th의 배수
                if ms.shape[-1] % self.th != 0: # wav길이 != th의 배수
                    num_splits += 1
                self.iter_list.append(num_splits)

                for i in range(num_splits):
                    start_idx = i * self.th
                    end_idx = (i + 1) * self.th
                    split = ms[..., start_idx:end_idx]

                    # th보다 길이가 짧다면
                    if split.shape[-1] < self.th:
                        # Resizing
                        if self.resizing is True:
                            scale = self.th / split.shape[-1]
                            target_shape = (split.shape[-2], self.th)
                            split = self.resize_spectrogram(split, target_shape)
                        # Padding
                        else:
                            split = self.padding(split, self.th)
                    # 최종 Resizing
                    resized = self.resize_spectrogram(split, self.target_size)
                    resized = self.normalize_spectrogram2(resized)  # 정규화
                    audio_list.append(resized)
                    if self.resizing is True:
                        self.scale_list.append(scale)

            # 원본 wav의 길이가 th보다 짧거나 같다면
            else:
                self.iter_list.append(1)
                scale = 1
                # th보다 짧다면
                if ms.shape[-1] < self.th:
                    # Resizing
                    if self.resizing is True:
                        scale = self.th / ms.shape[-1]
                        target_shape = (ms.shape[-2], self.th)
                        ms = self.resize_spectrogram(ms, target_shape)
                    # Padding
                    else:
                        ms = self.padding(ms, self.th)
                # 최종 resizing
                ms = self.resize_spectrogram(ms, self.target_size)
                ms = self.normalize_spectrogram2(ms)    # 정규화
                audio_list.append(ms)
                if self.resizing is True:
                    self.scale_list.append(scale)
        return torch.stack(audio_list)

    def get_label(self):
        labels = []
        idx=0
        for i, path_tsv in enumerate(self.tsvs):
            path = os.path.join(PATH, path_tsv)
            tsv_data = pd.read_csv(path, sep='\t', header=None)
            iter = self.iter_list[i]
            continuous = False
            next_end = 0
            next_class = 0
            for _iter in range(iter):
                label = []
                if self.resizing is True:
                    scale = self.scale_list[sum(self.iter_list[:i]) + _iter]
                for _, tsv_row in tsv_data.iterrows():
                    # 이전 구간에서 이어진다면 tsv_row[0] = 0
                    if continuous is True:
                        tsv_row[0] = 0
                        tsv_row[1] = next_end
                        tsv_row[2] = next_class
                        continuous = False
                        # resize를 하였다면 라벨 값도 스케일링
                        if self.resizing is True:
                            tsv_row[0] *= scale
                            tsv_row[1] *= scale

                    # 이전 구간에서 이어지지 않는다면 새로 데이터 가져오기
                    elif tsv_row[2] in [1, 3]:
                        # 구간 불러와서 sr값 곱하고 hop_legth로 나누기
                        tsv_row[0] = tsv_row[0] * SAMPLE_RATE / HOP_LENGTH - (_iter * self.th)
                        tsv_row[1] = tsv_row[1] * SAMPLE_RATE / HOP_LENGTH - (_iter * self.th)
                        tsv_row[2] = 0 if tsv_row[2] == 1 else 1    # S1=0, S2=1
                        # 시작점이 th 이상이라면, 이전 구간 이미지의 라벨이라면 continue
                        if tsv_row[0] >= self.th or tsv_row[0] < 0:
                            continue
                        # th보다 길이가 짧다면(나뉜 이미지의 가장 마지막 이미지라면)
                        if _iter == iter - 1:
                            # resize를 하였다면 라벨 값도 스케일링
                            if self.resizing is True:
                                tsv_row[0] *= scale
                                tsv_row[1] *= scale

                        # th보다 길이가 길다면 Slicing
                        elif tsv_row[1] >= self.th:
                            next_end = tsv_row[1] - self.th
                            next_class = tsv_row[2]
                            tsv_row[1] = self.th # 지움
                            continuous = True
                            # 최종 resize한 값 으로 보간
                            tsv_row[0] *= self.target_size[1] / self.th
                            tsv_row[1] *= self.target_size[1] / self.th

                            # S1=1, S2=2
                            label.append([tsv_row[0] / self.target_size[0], 0 / self.target_size[0],
                                  tsv_row[1] / self.target_size[0], self.target_size[0] / self.target_size[0],
                                  int(tsv_row[2])+1])# xmin, ymin, xmax, ymax, cls
                            break

                    # 이전 값에서 이어지지 않으면서 S1, S2가 아닌 경우 continue
                    else: continue

                    # 최종 resize한 값 으로 보간
                    tsv_row[0] *= self.target_size[1] / self.th
                    tsv_row[1] *= self.target_size[1] / self.th

                    label.append([tsv_row[0] / self.target_size[0], 0 / self.target_size[0],
                                  tsv_row[1] / self.target_size[0], self.target_size[0] / self.target_size[0],
                                  int(tsv_row[2])+1])# xmin, ymin, xmax, ymax, cls

                    #label.append((int(tsv_row[2]), tsv_row[0], tsv_row[1]))
                if(len(label)==0):
                    self.delete_list.append(idx)
                idx+=1
                labels.append(label)
        return labels

    def delete_data(self):
        delete_count=0
        for i in self.delete_list:
            del self.y[i-delete_count]
            delete_count+=1

        self.x = self.x[[i for i in range(self.x.size(0)) if i not in self.delete_list]]



In [ ]:
dataset = CustomDataset(PATH, train_patient_txt, target_size=(300, 300), th=5, resizing=True, filtering=True)

In [ ]:
len(dataset.delete_list)
dataset.delete_list
print(len(dataset.y),len(dataset.x))
dataset.x.size(0)

# data loader 생성

In [ ]:
from torch.utils.data import DataLoader

def my_collate_fn(batch):

    targets = []
    imgs = []
    for sample in batch:
        imgs.append(sample[0])  # sample[0]은 화상 gt
        targets.append(torch.FloatTensor(sample[1]))  # sample[1]은 어노테이션 gt

    imgs = torch.stack(imgs, dim=0)
    return imgs, targets

train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=my_collate_fn)

# model 생성

In [11]:
from torchvision import models
import torch
device = torch.device("cuda")

backbone = models.mobilenet_v3_large(pretrained=True)

#for param in backbone.parameters():
#    param.requires_grad = False

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V3_Large_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V3_Large_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-8738ca79.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-8738ca79.pth
100%|██████████| 21.1M/21.1M [00:00<00:00, 76.6MB/s]


In [12]:
for name, module in backbone.named_children():
    print(name)

for n, x in enumerate(backbone.features.children()):
    if n==0:
        seq=x.children()
        seq=next(seq)

        prev_weight = seq.weight
        new_weight = prev_weight[:, :1, :, :]
        seq.weight = nn.Parameter(new_weight)
        seq.in_channels = 1
    if n==13:
        seq=x.children()
        seq=next(seq)
        print(seq[0])

features
avgpool
classifier
Conv2dNormActivation(
  (0): Conv2d(112, 672, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(672, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (2): Hardswish()
)


In [13]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

class SSD(nn.Module):
    def __init__(self, backbone, n_class=3, default_box_n=[4,6,6,6,4,4], state = "Train"):

        super().__init__()

        self.input_layer = nn.Conv2d(1, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))

        self.n_class = n_class
        self.default_box_n = default_box_n
        self.state = state

        self.softmax = nn.Softmax(dim=-1)

        #가중치 초기화 인자
        self.rescale_factors = nn.Parameter(torch.FloatTensor(1, 112, 1, 1))  # there are 512 channels in conv4_3_feats
        nn.init.constant_(self.rescale_factors, 20)

        self.relu = nn.ReLU()
        #backbone

        self.backbone_layer = nn.Sequential(
            backbone.features,
        )

        #extra layer
        self.extra_layer_1 = self.extra_layers(960,512,4)
        self.extra_layer_2 = self.extra_layers(512,256,4)
        self.extra_layer_3 = self.extra_layers(256,256,2)
        self.extra_layer_4 = self.extra_layers(256,128,2)

        self.extra_layers = [self.extra_layer_1,
                            self.extra_layer_2,
                            self.extra_layer_3,
                            self.extra_layer_4]


        #detection output

        self.cls_layers = nn.ModuleList([
                                        nn.Conv2d(672, default_box_n[0]*(self.n_class), kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                                        nn.Conv2d(960, default_box_n[1]*(self.n_class), kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                                        nn.Conv2d(512, default_box_n[2]*(self.n_class), kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                                        nn.Conv2d(256, default_box_n[3]*(self.n_class), kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                                        nn.Conv2d(256, default_box_n[4]*(self.n_class), kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                                        nn.Conv2d(128, default_box_n[5]*(self.n_class), kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                                        ])

        self.loc_layers = nn.ModuleList([
                                        nn.Conv2d(672, default_box_n[0]*4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                                        nn.Conv2d(960, default_box_n[1]*4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                                        nn.Conv2d(512, default_box_n[2]*4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                                        nn.Conv2d(256, default_box_n[3]*4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                                        nn.Conv2d(256, default_box_n[4]*4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                                        nn.Conv2d(128, default_box_n[5]*4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
                                        ])



    def init_conv2d(self):
        #가중치 초기화 함수

        #모델학습이 순조롭지않다면 향후 추가 예정
        return

    def extra_layers(self, input_size, output_size, div):
        layer = nn.Sequential(
            #conv2D 해상도낮추기
            nn.Conv2d(input_size, output_size, kernel_size=(1, 1), stride=(1, 1), bias=False),
            nn.BatchNorm2d(output_size, eps=0.001, momentum=0.01, affine=True, track_running_stats=True),
            nn.Hardswish(),

            #Inverted Residual (mobilev2 + squeeze)

            #depthwise
            # kernel size 5 고려해보기
            nn.Conv2d(output_size, output_size, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=output_size, bias=False),
            nn.BatchNorm2d(output_size, eps=0.001, momentum=0.01, affine=True, track_running_stats=True),
            nn.Hardswish(),

            #SqueezeExcitation
            nn.Conv2d(output_size, output_size//div, kernel_size=(1, 1), stride=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(output_size//div, output_size, kernel_size=(1, 1), stride=(1, 1)),

            #Point-wise
            nn.Conv2d(output_size, output_size, kernel_size=(1, 1), stride=(1, 1)),
            nn.Hardswish(),
            nn.Identity()

        )
        return layer

    def forward(self, x):
        f_maps=[]
        for n, layer in enumerate(backbone.features):#12 16
            if n==13:
                #L2 norm
                #norm = x.pow(2).sum(dim=1, keepdim=True).sqrt()     # (N, 1, 19, 19)
                #featureMap_1 = x / norm                                  # (N, 112, 19, 19)
                #featureMap_1= conv4_3 * self.rescale_factors            # (N, 512, 19, 19)

                #13계층 bottleneck까지만
                seq_layer = next(layer.children())
                for seq_n in range(4):
                    x = seq_layer[seq_n](x)
                    if seq_n==0:
                        f_maps.append(x)
                continue

            x = layer(x)
            #print("size : {0}, number = {1}".format(x.size(), n))
            if n==16:
                f_maps.append(x)

        #extra layer
        mini_batch_n = x.size(0)
        for extra_layer in self.extra_layers:
            for idx, layer in enumerate(extra_layer.children()):
                if mini_batch_n ==1 and (idx == 1 or idx == 4):
                    continue
                x = layer(x)

            f_maps.append(x)

        cls = []
        loc = []
        for f_map, cls_layer, loc_layer in zip(f_maps,self. cls_layers, self.loc_layers):
            output_cls = cls_layer(f_map)
            output_loc = loc_layer(f_map)

            cls.append(output_cls.permute(0, 2, 3, 1).contiguous())
            loc.append(output_loc.permute(0, 2, 3, 1).contiguous())
            #cls.append(output_cls)
            #loc.append(output_loc)

        cls = torch.cat([o.view(o.size(0), -1) for o in cls], 1)
        loc = torch.cat([o.view(o.size(0), -1) for o in loc], 1)

        loc = loc.view(loc.size(0), -1, 4)
        if self.state == "Train":
            #cls = cls.view(cls.size(0), -1, self.n_class)
            cls = self.softmax(cls.view(cls.size(0), -1, self.n_class))
        else:
            cls = self.softmax(cls.view(cls.size(0), -1, self.n_class))


        return cls, loc

# default box, loss 함수 기타 기능 구현

In [14]:
import torch


def point_form(boxes):
    return torch.cat((boxes[:, :2] - boxes[:, 2:]/2,     # xmin, ymin
                     boxes[:, :2] + boxes[:, 2:]/2), 1)  # xmax, ymax


def center_size(boxes):
    return torch.cat((boxes[:, 2:] + boxes[:, :2])/2,  # cx, cy
                     boxes[:, 2:] - boxes[:, :2], 1)  # w, h


def intersect(box_a, box_b):
    A = box_a.size(0) #정답개수
    B = box_b.size(0) #defalut 박스개수
    max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(A, B, 2),
                       box_b[:, 2:].unsqueeze(0).expand(A, B, 2)) #정답, dbox , xmax, ymax
    min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(A, B, 2),
                       box_b[:, :2].unsqueeze(0).expand(A, B, 2))
    inter = torch.clamp((max_xy - min_xy), min=0)
    return inter[:, :, 0] * inter[:, :, 1]


def jaccard(box_a, box_b):# iou
    inter = intersect(box_a, box_b) # (target_n,dbox_n)
    # 확인햇음
    area_a = ((box_a[:, 2]-box_a[:, 0]) *
              (box_a[:, 3]-box_a[:, 1])).unsqueeze(1).expand_as(inter)  # [A,B]
    area_b = ((box_b[:, 2]-box_b[:, 0]) *
              (box_b[:, 3]-box_b[:, 1])).unsqueeze(0).expand_as(inter)  # [A,B]
    union = area_a + area_b - inter
    return inter / union  # [A,B]


def match(threshold, truths, priors, variances, labels, loc_t, conf_t, idx):
    #groud truth =target
    # jaccard index
    # defuatl box와 truths box iou 계산
    overlaps = jaccard(
        truths,
        point_form(priors)
    )

    #overlaps.size() = target_n, dbox_n

    # target기준 가장 성능이 좋은 dbox 값 검출, dbox 인덱스 검출
    best_prior_overlap, best_prior_idx = overlaps.max(1, keepdim=True) #차원, 형태 삭제 x size = target,1

    # dbox 기준 가장 성능이 높았던 target 검출, target 인덱스 검출
    best_truth_overlap, best_truth_idx = overlaps.max(0, keepdim=True) #size = 1, bbox

    #차원 제거
    best_truth_idx.squeeze_(0)
    best_truth_overlap.squeeze_(0)
    best_prior_idx.squeeze_(1)
    best_prior_overlap.squeeze_(1)
    #print(best_truth_overlap, best_truth_idx)
    #dbox기준 target값을 dbox 인덱스에 해당하면 값을 2로 변경
    best_truth_overlap.index_fill_(0, best_prior_idx, 2)  # 0차원에  best_prior_idx 위치에 value 2를 채움
    #1, bbox     fill = target,1 ?????
    for j in range(best_prior_idx.size(0)):
        #타겟별 가장 성능이 좋은 dbox의 index 추출
        #해당 index의 ddox의 가장 성능이 좋았던 타겟값으로 변경 - 안해도 되는작업이아닌가?
        best_truth_idx[best_prior_idx[j]] = j

    #print(truths.size())
    matches = truths[best_truth_idx]          # Shape: [num_priors,4] #dbox 번호별로 가장 어울리는 타겟값의 정보를 저장함
    conf = labels[best_truth_idx]#+1         # Shape: [num_priors] # dbox 번호별로 가장 어울리는 타겟값의 라벨을 저장함
    conf[best_truth_overlap < threshold] = 0  # 실제 target값이더라도 dbox들의 iou 값이 낮다면 배경으로 판단

    #해당코드에서 문제가 발생된다고 예상
    loc = encode(matches, priors, variances, truths)

    # idx 는 미니 배치 index
    loc_t[idx] = loc    # [num_priors,4] encoded offsets to learn
    conf_t[idx] = conf  # [num_priors] top class label for each prior


def encode(matched, priors, variances, truths):
    #torch.autograd.set_detect_anomaly(True)

    # dist b/t match center and prior's center
    g_cxcy = (matched[:, :2] + matched[:, 2:])/2 - priors[:, :2]
    # encode variance
    g_cxcy /= (variances[0] * priors[:, 2:])

    #print("****************************************************")
    eps = 1e-4
    # match wh / prior wh
    g_wh = (matched[:, 2:] - matched[:, :2]) / priors[:, 2:]
    g_wh = torch.log(g_wh+eps) / variances[1]

    #print("****************************************************")
    return torch.cat([g_cxcy, g_wh], 1)  # [num_priors,4]


# Adapted from https://github.com/Hakuyume/chainer-ssd
def decode(loc, priors, variances):
    boxes = torch.cat((
        priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],
        priors[:, 2:] * torch.exp(loc[:, 2:] * variances[1])), 1)
    boxes[:, :2] -= boxes[:, 2:] / 2
    boxes[:, 2:] += boxes[:, :2]
    return boxes


def log_sum_exp(x):
    x_max = x.data.max()
    return torch.log(torch.sum(torch.exp(x-x_max), 1, keepdim=True)) + x_max


def nms(boxes, scores, overlap=0.5, top_k=200):
    keep = scores.new(scores.size(0)).zero_().long()
    if boxes.numel() == 0:
        return keep
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    area = torch.mul(x2 - x1, y2 - y1)
    v, idx = scores.sort(0)  # sort in ascending order
    # I = I[v >= 0.01]
    idx = idx[-top_k:]  # indices of the top-k largest vals
    xx1 = boxes.new()
    yy1 = boxes.new()
    xx2 = boxes.new()
    yy2 = boxes.new()
    w = boxes.new()
    h = boxes.new()

    # keep = torch.Tensor()
    count = 0
    while idx.numel() > 0:
        i = idx[-1]  # index of current largest val
        # keep.append(i)
        keep[count] = i
        count += 1
        if idx.size(0) == 1:
            break
        idx = idx[:-1]  # remove kept element from view
        # load bboxes of next highest vals
        torch.index_select(x1, 0, idx, out=xx1)
        torch.index_select(y1, 0, idx, out=yy1)
        torch.index_select(x2, 0, idx, out=xx2)
        torch.index_select(y2, 0, idx, out=yy2)
        # store element-wise max with next highest score
        xx1 = torch.clamp(xx1, min=x1[i])
        yy1 = torch.clamp(yy1, min=y1[i])
        xx2 = torch.clamp(xx2, max=x2[i])
        yy2 = torch.clamp(yy2, max=y2[i])
        w.resize_as_(xx2)
        h.resize_as_(yy2)
        w = xx2 - xx1
        h = yy2 - yy1
        # check sizes of xx1 and xx2.. after each iteration
        w = torch.clamp(w, min=0.0)
        h = torch.clamp(h, min=0.0)
        inter = w*h
        # IoU = i / (area(a) + area(b) - i)
        rem_areas = torch.index_select(area, 0, idx)  # load remaining areas)
        union = (rem_areas - inter) + area[i]
        IoU = inter/union  # store result in iou
        # keep only elements with an IoU <= overlap
        idx = idx[IoU.le(overlap)]
    return keep, count

In [15]:
import torch.nn.functional as F

class MultiBoxLoss(nn.Module):
    """SSD의 손실함수 클래스 """

    def __init__(self, thresh=0.5, neg_pos=3, device='cpu'):
        super(MultiBoxLoss, self).__init__()
        self.jaccard_thresh = thresh  # 0.5 match 함수의 jaccard 계수의 임계치
        self.negpos_ratio = neg_pos  # 3:1 Hard Negative Mining의 음과 양 비율
        self.device = device  # 계산 device(CPU | GPU)

    def forward(self, predictions, targets, dboxs):
        """
        파라미터 설명
        ----------
        predictions :모델의 예측값 cls와 loc
        cls는 batch dbox의 개수, 클래스 개수로 이루어짐
        loc은 batch dbox의 개수, 4

        targets : [num_batch, 객체개수, 5]
            5는 라벨 정보[xmin, ymin, xmax, ymax, label_ind]

        """

        conf_data, loc_data = predictions
        dbox_list = dboxs

        num_batch = loc_data.size(0)  # 배치 크기
        num_dbox = loc_data.size(1)  # DBox의 수
        num_classes = conf_data.size(2)  # 클래스 수

        # 손실 계산에 사용할 것을 저장하는 변수 작성
        # conf_t_label：각 DBox에 가장 가까운 정답 BBox의 라벨을 저장
        # loc_t: 각 DBox에 가장 가까운 정답 BBox의 위치 정보 저장
        conf_t_label = torch.LongTensor(num_batch, num_dbox).to(self.device)

        #conf_t_label.fill_(0)#테스트용도
        loc_t = torch.Tensor(num_batch, num_dbox, 4).to(self.device)

        # loc_t와 conf_t_label에
        # DBox와 정답 어노테이션 targets를 amtch한 결과 덮어쓰기
        for idx in range(num_batch):  # 미니 배치 루프

            truths = targets[idx][:, :-1].to(self.device)
            labels = targets[idx][:, -1].to(self.device)
            dbox = dbox_list.to(self.device)

            # match 함수를 실행하여 loc_t와 conf_t_label 내용 갱신
            # loc_t: 각 DBox에 가장 가까운 정답 BBox 위치 정보가 덮어써짐.
            # conf_t_label：각 DBox에 가장 가까운 정답 BBox 라벨이 덮어써짐.
            # 단, 가장 가까운 BBox와 iou가 0.5보다 작은 경우,
            # 정답 BBox의 라벨 conf_t_label은 배경 클래스 0으로 한다.
            variance = [0.1, 0.2]

            # 라벨을 dbox에 대한 offset으로 변환
            match(self.jaccard_thresh, truths, dbox,
                  variance, labels, loc_t, conf_t_label, idx)


        #물체를 발견한 offset만 손실 계산
        pos_mask = conf_t_label > 0  # size: batch,dbox,1
        true_count = pos_mask[0].sum().item()

        # pos_mask를 loc_data 크기로 변형
        pos_idx = pos_mask.unsqueeze(pos_mask.dim()).expand_as(loc_data)

        # Positive DBox의 loc_data와 offset loc_t 취득
        loc_p = loc_data[pos_idx].view(-1, 4).to(device)
        loc_t = loc_t[pos_idx].view(-1, 4)

        # 물체를 발견한 Positive DBox의 오프셋 정보 loc_t의 손실(오차)를 계산
        loss_l = F.smooth_l1_loss(loc_p, loc_t, reduction='sum')
        # ----------
        # 클래스 예측의 손실 : loss_c를 계산
        # 교차 엔트로피 오차 함수로 손실 계산. 단 배경 클래스가 정답인 DBox가 압도적으로 많으므로,
        # Hard Negative Mining을 실시하여 물체 발견 DBox 및 배경 클래스 DBox의 비율이 1:3이 되도록 한다.
        # 배경 클래스 DBox로 예상한 것 중 손실이 적은 것은 클래스 예측 손실에서 제외
        # ----------
        batch_conf = conf_data.view(-1, num_classes)

        # 클래스 예측의 손실함수 계산(reduction='none'으로 하여 합을 취하지 않고 차원 보존)
        loss_c = F.cross_entropy(
            batch_conf, conf_t_label.view(-1), reduction='none')#batch * dbox_n


        #------------이 아래 부분은 이해 못함



        # -----------------
        # Negative DBox중 Hard Negative Mining으로
        # 추출하는 것을 구하는 마스크 작성
        # -----------------

        # 물체를 발견한 Positive DBox의 손실을 0으로 한다.
        # (주의) 물체는 label이 1 이상, 라벨 0은 배경을 의미
        num_pos = pos_mask.long().sum(1, keepdim=True)  # 미니 배치별 물체 클래스 예측 수
        loss_c = loss_c.view(num_batch, -1)  # torch.Size([num_batch, 8732])
        loss_c[pos_mask] = 0  # 물체를 발견한 DBox는 손실 0으로 한다.

        # Hard Negative Mining
        # 각 DBox 손실의 크기 loss_c 순위 idx_rank를 구함
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)

        #  (주의) 구현된 코드는 특수하며 직관적이지 않음.
        # 위 두 줄의 요점은 각 DBox에 대해 손실 크기가 몇 번째인지의 정보를
        # idx_rank 변수로 빠르게 얻는 코드이다.

        # DBox의 손실 값이 큰 쪽부터 내림차순으로 정렬하여,
        # DBox의 내림차순의 index를 loss_idx에 저장한다.
        # 손실 크기 loss_c의 순위 idx_rank를 구한다.
        # 내림차순이 된 배열 index인 loss_idx를 0부터 8732까지 오름차순으로 다시 정렬하기 위하여
        # 몇 번째 loss_idx의 인덱스를 취할 지 나타내는 것이 idx_rank이다.
        # 예를 들면 idx_rank 요소의 0번째 = idx_rank[0]을 구하는 것은 loss_idx의 값이 0인 요소,
        # 즉 loss_idx[?] =0은 원래 loss_c의 요소 0번째는 내림차순으로 정렬된 loss_idx의
        # 몇 번째입니까? 를구하는 것이 되어 결과적으로,
        # ? = idx_rank[0]은 loss_c의 요소 0번째가 내림차순으로 몇 번째인지 나타냄

        # 배경 DBox의 수 num_neg를 구한다. HardNegative Mining으로
        # 물체 발견 DBox으 ㅣ수 num_pos의 세 배 (self.negpos_ratio 배)로 한다.
        # DBox의 수를 초과한 경우에는 DBox의 수를 상한으로 한다.
        num_neg = torch.clamp(num_pos*self.negpos_ratio, max=num_dbox)

        # idx_rank에 각 DBox의 손실 크기가 위에서부터 몇 번째인지 저장되었다.
        # 배경 DBox의 수 num_neg보다 순위가 낮은(손실이 큰) DBox를 취하는 마스크 작성
        # torch.Size([num_batch, 8732])
        neg_mask = idx_rank < (num_neg).expand_as(idx_rank)

        # -----------------
        # (종료) 지금부터 Negative DBox 중 Hard Negative Mining으로 추출할 것을 구하는 마스크를 작성
        # -----------------

        # 마스크 모양을 고쳐 conf_data에 맞춘다
        # pos_idx_mask는 Positive DBox의 conf를 꺼내는 마스크이다.
        # neg_idx_mask는 Hard Negative Mining으로 추출한 Negative DBox의 conf를 꺼내는 마스크이다.
        # pos_mask：torch.Size([num_batch, 8732])
        # --> pos_idx_mask：torch.Size([num_batch, 8732, 21])
        pos_idx_mask = pos_mask.unsqueeze(2).expand_as(conf_data)
        neg_idx_mask = neg_mask.unsqueeze(2).expand_as(conf_data)

        # conf_data에서 pos와 neg만 꺼내서 conf_hnm으로 한다.
        # 형태는 torch.Size([num_pos+num_neg, 21])
        conf_hnm = conf_data[(pos_idx_mask+neg_idx_mask).gt(0)
                             ].view(-1, num_classes)
        # gt는 greater than (>)의 약칭. mask가 1인 index를 꺼낸다
        # pos_idx_mask+neg_idx_mask는 덧셈이지만 index로 mask를 정리할 뿐임.
        # pos이든 neg이든 마스크가 1인 것을 더해 하나의 리스트로 만들어 이를 gt로 췯그한다.

        # 마찬가지로 지도 데이터인 conf_t_label에서 pos와 neg만 꺼내, conf_t_label_hnm 으로
        # torch.Size([pos+neg]) 형태가 된다
        conf_t_label_hnm = conf_t_label[(pos_mask+neg_mask).gt(0)]

        # confidence의 손실함수 계산(요소의 합계=sum을 구함)
        loss_c = F.cross_entropy(conf_hnm, conf_t_label_hnm, reduction='sum')

        # 물체를 발견한 BBox의 수 N (전체 미니 배치의 합계) 으로 손실을 나눈다.
        N = num_pos.sum()
        loss_l /= N
        loss_c /= N

        #print("-"*100)

        return loss_l, loss_c

In [16]:
import math
from math import sqrt
from itertools import product as product

class default:

    def __init__(self, image_size=300, feature_maps=[19, 10, 5, 3, 2, 1], min_sizes=[]):
        super(default, self).__init__()
        self.image_size = 300
        # number of priors for feature map location (either 4 or 6)
        self.feature_maps = feature_maps
        self.min_sizes = [16, 30, 60, 100, 150, 300]  #0.2, 0.34, 0.48, 0.62, 0.76, 0.9?
        self.max_sizes = [30, 60, 100, 150, 300, 300]
        self.steps = [19,10,5,3,2,1] # 이미지 그리드로 나눈 개수
        self.aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
        self.clip = True

    def forward(self):
        mean = []
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.steps[k] # 그리드 개수
                # default 박스 중점
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k
                # aspect_ratio: 1
                # default 박스 공식이 아닌 임의로 설정 그리드 크기를 기준으로 나눔
                s_k = self.min_sizes[k]/self.image_size
                mean += [cx, cy, s_k, 0.7] # s_k

                # aspect_ratio: 1
                # s_k*s_k+1
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))

                #print(cx,cy,s_k_prime,self.max_sizes[k]/self.image_size)
                mean += [cx, cy, s_k_prime, 0.7] #s_k

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    mean += [cx, cy, s_k*sqrt(ar), 0.7] # s_k/sqrt(ar)
                    mean += [cx, cy, s_k/sqrt(ar), 0.7]
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output
    def cxcy_to_xy(self, tensor_default_box):
        return tensor_default_box*self.image_size

    def xy_to_cxcy(self, tensor_default_box):
        return tensor_default_box/self.image_size

In [ ]:
backbone = models.mobilenet_v3_large(pretrained=True)
for name, module in backbone.named_children():
    print(name)

for n, x in enumerate(backbone.features.children()):
    if n==0:
        seq=x.children()
        seq=next(seq)

        prev_weight = seq.weight
        new_weight = prev_weight[:, :1, :, :]
        seq.weight = nn.Parameter(new_weight)
        seq.in_channels = 1
    if n==13:
        seq=x.children()
        seq=next(seq)
        print(seq[0])
model = SSD(backbone, n_class=3)
model = model.to(device)

features
avgpool
classifier
ConvBNActivation(
  (0): Conv2d(112, 672, kernel_size=(1, 1), stride=(1, 1), bias=False)
  (1): BatchNorm2d(672, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
  (2): Hardswish()
)


In [ ]:
train_step(model, train_dataloader, epoch_num = 10)
4, 5

Epoch : 1 / 10
Current Batch 0 / 132 | Cls Loss : 6.894, Loc Loss : 5.200, Total Loss : 12.094 | 10 iter time 0.3998: 
Current Batch 10 / 132 | Cls Loss : 4.164, Loc Loss : 4.701, Total Loss : 8.865 | 10 iter time 0.3167: 
Current Batch 20 / 132 | Cls Loss : 2.876, Loc Loss : 4.646, Total Loss : 7.522 | 10 iter time 0.3187: 
Current Batch 30 / 132 | Cls Loss : 3.020, Loc Loss : 4.579, Total Loss : 7.598 | 10 iter time 0.3185: 
Current Batch 40 / 132 | Cls Loss : 2.905, Loc Loss : 4.521, Total Loss : 7.427 | 10 iter time 0.3113: 
Current Batch 50 / 132 | Cls Loss : 2.624, Loc Loss : 4.488, Total Loss : 7.112 | 10 iter time 0.3453: 
Current Batch 60 / 132 | Cls Loss : 2.897, Loc Loss : 4.480, Total Loss : 7.377 | 10 iter time 0.3114: 
Current Batch 70 / 132 | Cls Loss : 2.678, Loc Loss : 4.466, Total Loss : 7.144 | 10 iter time 0.3124: 
Current Batch 80 / 132 | Cls Loss : 2.659, Loc Loss : 4.464, Total Loss : 7.122 | 10 iter time 0.3190: 
Current Batch 90 / 132 | Cls Loss : 3.245, Loc Lo

(4, 5)

In [ ]:
for idx, data in enumerate(train_dataloader):
    print(idx)
    print(data[1][0][:,-1])
    break

In [ ]:
len(train_dataloader)
print(os.getcwd())

In [ ]:
print(test1)
print(test2)

tensor([[-0.0063,  1.4286],
        [ 0.3607,  1.4286],
        [ 0.3492,  1.4286],
        ...,
        [ 0.0382,  1.4286],
        [ 0.0382,  1.4286],
        [ 0.0540,  1.4286]], device='cuda:0')
tensor([[     nan,   1.7837],
        [ -5.0973,   1.7837],
        [ -5.2586,   1.7837],
        ...,
        [-16.3142,   1.7837],
        [-16.3142,   1.7837],
        [-14.5852,   1.7837]], device='cuda:0')


In [ ]:
print(test1)
print(test2)
#test1 + 1e-4
print(torch.log(test1 + 1e-4) / 0.2)

tensor([[-0.0114,  1.4286],
        [ 0.1660,  1.4286],
        [ 0.1608,  1.4286],
        ...,
        [ 0.0155,  1.4286],
        [ 0.0155,  1.4286],
        [ 0.0220,  1.4286]], device='cuda:0')
tensor([[     nan,   1.7837],
        [ -8.9746,   1.7837],
        [ -9.1359,   1.7837],
        ...,
        [-20.7889,   1.7837],
        [-20.7889,   1.7837],
        [-19.0654,   1.7837]], device='cuda:0')
tensor([[     nan,   1.7837],
        [ -8.9746,   1.7837],
        [ -9.1359,   1.7837],
        ...,
        [-20.7889,   1.7837],
        [-20.7889,   1.7837],
        [-19.0654,   1.7837]], device='cuda:0')


In [ ]:
torch.log(torch.tensor(-0.0063))/0.2

tensor(nan)

In [ ]:
print(test3)

tensor([[0.5455, 0.0000, 0.5577, 1.0000],
        [0.5836, 0.0000, 0.5974, 1.0000],
        [0.6268, 0.0000, 0.6414, 1.0000],
        [0.6701, 0.0000, 0.6847, 1.0000],
        [0.7076, 0.0000, 0.7211, 1.0000],
        [0.7498, 0.0000, 0.7619, 1.0000],
        [0.7861, 0.0000, 0.8002, 1.0000],
        [0.8367, 0.0000, 0.8523, 1.0000],
        [0.8729, 0.0000, 0.8848, 1.0000],
        [0.9110, 0.0000, 0.9214, 1.0000],
        [0.9598, 0.0000, 0.9724, 1.0000],
        [0.9986, 0.0000, 0.9980, 1.0000]], device='cuda:0')


# **1.Test 평가 모드**

In [64]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader


# 테스트 데이터 로더 설정
train_dataloader = DataLoader(test_loader, batch_size=8, shuffle=False)


# 학습 반복
num_epochs = 10
test_interval = 20  # 테스트 간격 설정

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # 훈련 코드...?

        running_loss += loss.item()

        # 테스트 코드 (test_interval마다 실행)
        if (i + 1) % test_interval == 0:
            model.eval()  # 모델을 평가 모드로 전환

            test_loss = 0.0
            all_predictions = []  # 모든 예측을 저장할 리스트
            all_labels = []  # 모든 실제 레이블을 저장할 리스트

            with torch.no_grad():  # 그라디언트 계산 비활성화
                for inputs, labels in test_loader:
                    outputs = model(inputs)
                    test_loss += criterion(outputs, labels).item()

                    _, predicted = torch.max(outputs, 1)
                    all_predictions.extend(predicted.tolist())
                    all_labels.extend(labels.tolist())

            test_loss /= len(test_loader)

            # 여기에서 mAP 계산 및 출력
            # 예측과 레이블을 사용하여 mAP를 계산하는 추가 코드 필요

            def calculate_mAP(model, test_loader, num_classes, device):
              model.eval()  # 모델을 평가 모드로 설정
              all_predictions = []
              all_labels = []

              with torch.no_grad():
                for inputs, labels in test_loader:
                  inputs = inputs.to(device)
                  outputs = model(inputs)
                  _, predicted = torch.max(outputs, 1)
                all_predictions.extend(predicted.cpu().numpy())
                all_labels.extend(labels.numpy())

                all_predictions = np.array(all_predictions)
                all_labels = np.array(all_labels)

    # 각 클래스에 대한 AP 계산
                average_precisions = []
                for class_id in range(num_classes):
                    class_mask = (all_labels == class_id)
                    class_predictions = all_predictions[class_mask]
                    class_labels = all_labels[class_mask]
                    true_positives = (class_predictions == class_labels).astype(np.float32)
                    false_positives = (class_predictions != class_labels).astype(np.float32)

                    num_true_positives = np.cumsum(true_positives)
                    num_false_positives = np.cumsum(false_positives)

                    recall = num_true_positives / np.maximum(
                        num_true_positives + num_false_positives, np.finfo(np.float64).eps)

                    precision = num_true_positives / np.maximum(
                        num_true_positives + num_false_positives, np.finfo(np.float64).eps)

        # AP 계산
                    average_precision = calculate_ap(recall, precision)
                    average_precisions.append(average_precision)

    # 모든 클래스의 AP의 평균을 계산하여 mAP를 얻음
                mAP = np.mean(average_precisions)
                return mAP


                def calculate_ap(recall, precision):

                  recall_thresholds = np.linspace(0, 1, 11)
                  precision_sum = 0
                  num_recall_thresholds = 0

                  for threshold in recall_thresholds:
                    mask = recall >= threshold

                    if np.any(mask):
                      precision_sum += np.max(precision[mask])
                      num_recall_thresholds += 1

                      average_precision = precision_sum / max(1, num_recall_thresholds) # 위의 calculate_mAP 함수를 사용하여 mAP를 계산
                      return average_precision

                      num_classes = conf_data.size(2)
                      device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU 사용 가능한 경우 CUDA로 설정
                      mAP = calculate_mAP(model, test_loader, num_classes, device)
                      print(f"mAP: {mAP}")

            print(f'Epoch {epoch + 1}, Iter {i + 1}, Train Loss: {running_loss / test_interval}, Test Loss: {test_loss}')

            # 모델을 다시 훈련 모드로 전환
            model.train()

NameError: ignored

# **2.mAP 계산 기능 추가**

*   calculate_mAP 함수를 사용하여 모델의 평균 정밀도 (mAP)를 계산하려면 모델의 예측과 실제 라벨을 사용하여 각 이미지에 대한 Average Precision (AP)을 계산하고 이를 평균화해야 함

코드 작성 단계




1.  이미지별로 예측과 실제 라벨을 저장할 리스트를 생성합니다.
2. 모델의 예측을 테스트 세트 이미지에 대해 수행하고 예측된 박스와 클래스 정보를 얻습니다.
3. 각 이미지에 대해 예측된 박스와 실제 라벨을 리스트에 추가합니다.
4. 모든 이미지에 대해 AP를 계산하고 이를 모두 저장합니다.
5. AP의 평균값을 계산하여 mAP를 얻습니다.







In [46]:
import torch
import numpy as np
from sklearn.metrics import average_precision_score

def calculate_mAP(model, test_loader):
    # 모델을 평가 모드로 설정
    model.eval()

    all_predictions = []  # 모든 이미지의 예측 정보 저장
    all_labels = []      # 모든 이미지의 실제 라벨 정보 저장

    with torch.no_grad():
        for images, targets in test_loader:
            # 이미지를 모델에 전달하여 예측 수행
            outputs = model(images)
            predictions = post_process(outputs)  # 모델의 출력을 후처리하여 예측된 박스 및 클래스 얻음

            # 예측과 실제 라벨을 리스트에 추가
            all_predictions.extend(predictions)
            all_labels.extend(targets)

    # AP를 계산하고 저장할 리스트
    ap_scores = []

    for i in range(len(all_labels)):
        # 각 이미지에 대한 예측과 실제 라벨 가져오기
        predictions = all_predictions[i]
        labels = all_labels[i]

        # AP 계산을 위해 실제 라벨을 이진 벡터로 변환
        true_labels = np.zeros(len(predictions))
        true_labels[labels] = 1

        # sklearn의 average_precision_score를 사용하여 AP 계산
        ap = average_precision_score(true_labels, predictions)
        ap_scores.append(ap)

    # 모든 이미지에 대한 AP의 평균을 계산하여 mAP 얻음
    mAP = np.mean(ap_scores)

    return mAP

# 모델 예측을 후처리하는 함수 (필요에 따라 수정)
def post_process(outputs):
    # 모델의 출력을 후처리하여 예측된 박스와 클래스를 반환
     (outputs, priors, conf_threshold=0.5, nms_threshold =0.5):
     confidences, locations = outputs
     num_classes = confidences.size(2)

    # 바운딩 박스 디코딩
    decoded_boxes = decode(locations, priors)

    # 각 클래스에 대해 NMS를 수행하고, confidence_threshold 이상인 예측만 유지
    all_predictions = []
    for class_idx in range(1, num_classes):
        class_scores = confidences[:, :, class_idx]  # 해당 클래스의 점수
        mask = class_scores > conf_threshold
        class_boxes = decoded_boxes[mask]
        class_scores = class_scores[mask]

        # NMS 적용
        keep = nms(class_boxes, class_scores, nms_threshold)
        class_boxes = class_boxes[keep]
        class_scores = class_scores[keep]

        # 예측된 바운딩 박스와 클래스 점수를 리스트에 추가
        predictions = torch.cat([class_boxes, class_scores.unsqueeze(1)], dim=1)
        all_predictions.append(predictions)

    if len(all_predictions) == 0:
        return torch.tensor([])

    # 모든 클래스의 예측을 하나의 텐서로 연결
    all_predictions = torch.cat(all_predictions, dim=0)

    return all_predictions

# 예측 후처리를 수행한 결과
predictions = post_process(outputs, priors, conf_threshold=0.5, nms_threshold=0.5)


# 모델과 테스트 데이터 로더를 사용하여 mAP 계산
mAP = calculate_mAP(model, test_loader)
print(f"Mean Average Precision (mAP): {mAP}")

# conf_threshold와 nms_threshold는 클래스 점수와 NMS의 임계값을 나타냄을 참고

IndentationError: ignored

##2-2.mAP 계산 (방법 2)
하단 코드로 도전하였고,

**-> 상단코드 안돌아가고, 하단은 돌아가네요.**





In [40]:
import torch
import numpy as np
from sklearn.metrics import average_precision_score

def calculate_mAP(model, test_dataset, device='cuda'):
    model.eval()
    with torch.no_grad():
        all_detections = []
        all_ground_truths = []
        for i in range(len(test_dataset)):
            image, targets = test_dataset[i]
            image = image.to(device)
            detections = model(image.unsqueeze(0))
            detections = detections.squeeze(0)

            # Detections를 박스의 좌표와 확률로 분리
            boxes = detections[:, :4]
            scores = detections[:, 4]

            # 정확도 기준으로 내림차순으로 정렬
            sorted_indices = scores.argsort(descending=True)
            boxes = boxes[sorted_indices]
            scores = scores[sorted_indices]

            # 실제 정답 정보
            true_boxes = targets[:, :4]
            true_labels = targets[:, 4]

            all_detections.append((boxes.cpu(), scores.cpu()))
            all_ground_truths.append((true_boxes.cpu(), true_labels.cpu()))

        # mAP 계산
        average_precisions = calculate_average_precisions(all_detections, all_ground_truths)

        # 모든 클래스에 대한 mAP 평균
        mean_ap = np.mean(average_precisions)

        return mean_ap

def calculate_average_precisions(all_detections, all_ground_truths):
    average_precisions = []
    for i in range(len(all_detections)):
        detections = all_detections[i]
        ground_truths = all_ground_truths[i]

        if len(ground_truths) == 0:
            continue

        detected_flags = [False] * len(ground_truths)
        num_detected = 0
        sorted_indices = np.argsort(-detections[1])

        precision_values = []
        recall_values = []

        for idx in sorted_indices:
            box = detections[0][idx]
            score = detections[1][idx]

            overlaps = calculate_overlaps(box.unsqueeze(0), ground_truths[0])
            max_overlap_idx = np.argmax(overlaps)

            if overlaps[max_overlap_idx] > 0.5 and not detected_flags[max_overlap_idx]:
                detected_flags[max_overlap_idx] = True
                num_detected += 1

            precision = num_detected / (idx + 1)
            recall = num_detected / len(ground_truths)

            precision_values.append(precision)
            recall_values.append(recall)

        average_precisions.append(calculate_area_under_curve(precision_values, recall_values))

    return average_precisions

def calculate_area_under_curve(x, y):
    sorted_indices = np.argsort(x)
    sorted_indices = sorted_indices[::-1]
    x = np.array(x)[sorted_indices]
    y = np.array(y)[sorted_indices]

    auc = 0
    prev_x = 0
    prev_y = 0

    for i in range(len(x)):
        width = x[i] - prev_x
        height = (y[i] + prev_y) / 2
        auc += width * height
        prev_x = x[i]
        prev_y = y[i]

    return auc

def calculate_overlaps(boxes1, boxes2):
    x1 = boxes1[:, 0]
    y1 = boxes1[:, 1]
    x2 = boxes1[:, 2]
    y2 = boxes1[:, 3]

    x1g = boxes2[:, 0]
    y1g = boxes2[:, 1]
    x2g = boxes2[:, 2]
    y2g = boxes2[:, 3]

    x1 = x1.unsqueeze(1).expand_as(x1g)
    y1 = y1.unsqueeze(1).expand_as(y1g)
    x2 = x2.unsqueeze(1).expand_as(x2g)
    y2 = y2.unsqueeze(1).expand_as(y2g)

    x1g = x1g.unsqueeze(0).expand_as(x1)
    y1g = y1g.unsqueeze(0).expand_as(y1)
    x2g = x2g.unsqueeze(0).expand_as(x2)
    y2g = y2g.unsqueeze(0).expand_as(y2)

    inter_x1 = torch.max(x1, x1g)
    inter_y1 = torch.max(y1, y1g)
    inter_x2 = torch.min(x2, x2g)
    inter_y2 = torch.min(y2, y2g)

    inter_width = torch.clamp(inter_x2 - inter_x1, min=0)
    inter_height = torch.clamp(inter_y2 - inter_y1, min=0)

    inter_area = inter_width * inter_height
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x2g - x1g) * (y2g - y1g)

    overlap = inter_area / (box1_area + box2_area - inter_area)

    return overlap



print(f"Mean Average Precision (mAP): {calculate_mAP}")


Mean Average Precision (mAP): <function calculate_mAP at 0x7947184c6b90>


# **3. 중심 좌표 x,y와 길이 w,h  (4개의 값)**

center_size 함수: 입력으로 주어진 바운딩 박스를 중심 좌표와 크기로 변환
decode 함수: 예측된 오프셋 정보와 DBox를 사용하여 원래 바운딩 박스를 계산

In [35]:
def center_size(boxes):
    return torch.cat((boxes[:, 2:] + boxes[:, :2]) / 2, boxes[:, 2:] - boxes[:, :2], 1)

def decode(loc, priors, variances):
    boxes = torch.cat((
        priors[:, :2] + loc[:, :2] * variances[0] * priors[:, 2:],
        priors[:, 2:] * torch.exp(loc[:, 2:] * variances[1])), 1)
    boxes[:, :2] -= boxes[:, 2:] / 2
    boxes[:, 2:] += boxes[:, :2]
    return boxes

print("중심 좌표 (x, y):", x, y)
print("너비 (w):", w)
print("높이 (h):", h)

중심 좌표 (x, y): tensor([-0.0004, -0.0102, -0.0114,  ...,  0.0000,  0.0000,  0.1464]) tensor([-0.3237, -0.3237, -0.3237,  ...,  0.1500,  0.1500,  0.1500])
너비 (w): tensor([0.0533, 0.0730, 0.0754,  ..., 1.0000, 1.0000, 0.7071])
높이 (h): tensor([0.7000, 0.7000, 0.7000,  ..., 0.7000, 0.7000, 0.7000])
